In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#reading all the datafiles
hospitals = pd.read_csv("../input/usa-hospitals/Hospitals.csv")
hospital_ratings = pd.read_csv("../input/hospital-ratings/Hospital General Information.csv", encoding="ISO-8859-1")
public_schools = pd.read_csv("../input/usa-public-schools/Public_Schools.csv")
county_time_series = pd.read_csv("../input/zecon/County_time_series.csv")
crosswalk = pd.read_csv("../input/zecon/CountyCrossWalk_Zillow.csv")
unemployment = pd.read_csv("../input/2018-unemployment-rate-by-county/GeoFRED_Unemployment_Rate_by_County_Percent.csv")


Merging all the datasets together to create a dataset that can give a basic idea of house prices according to the data features such as unemployment_rate, schools , hospitals etc.

In [ ]:
unemployment.head()

In [ ]:
hospitals.head()

In [ ]:
hospital_ratings.head()

In [ ]:
hospital_ratings = hospital_ratings[['Hospital Name', 'Hospital overall rating']]
hospital_ratings.dropna()
hospital_ratings.head()

In [ ]:
hospitals_with_ratings = pd.merge(hospitals, hospital_ratings, how='left', left_on='NAME', right_on='Hospital Name')

hospitals_with_ratings = hospitals_with_ratings.loc[hospitals_with_ratings['COUNTYFIPS'] != 'NOT AVAILABLE']

hospitals_with_ratings = hospitals_with_ratings.loc[~pd.isna(hospitals_with_ratings)['Hospital overall rating']]
hospitals_with_ratings = hospitals_with_ratings.loc[hospitals_with_ratings['Hospital overall rating'] != 'Not Available']
hospitals_with_ratings['Hospital overall rating'] = hospitals_with_ratings['Hospital overall rating'].astype("int64")

hospitals_with_ratings.rename(index=str, columns={"COUNTYFIPS": "FIPS", "Hospital overall rating": "AverageHospitalRating"}, inplace=True)

hospitals_with_ratings.head()

In [ ]:
unemployment_per_county = unemployment.loc[:,("Region Code", "2018")]
unemployment_per_county.rename(index=str, columns={"Region Code": "FIPS", "2018": "UnemploymentRate"}, inplace=True)
unemployment_per_county["FIPS"] = unemployment_per_county["FIPS"].astype("int64")

unemployment_per_county.head()

In [ ]:
average_hospital_rating_per_county = hospitals_with_ratings[['FIPS', 'AverageHospitalRating']]

average_hospital_rating_per_county = average_hospital_rating_per_county.dropna()

average_hospital_rating_per_county = average_hospital_rating_per_county.groupby(['FIPS']).mean()
average_hospital_rating_per_county.reset_index(level=0, inplace=True)
average_hospital_rating_per_county["FIPS"] = average_hospital_rating_per_county["FIPS"].astype("int64")

average_hospital_rating_per_county.head()

In [ ]:
hospitals_per_county = hospitals['COUNTYFIPS'].value_counts().to_frame()
hospitals_per_county.reset_index(level=0, inplace=True)
hospitals_per_county.rename(index=str, columns={"index": "FIPS", "COUNTYFIPS": "NumberOfHospitals"}, inplace=True)
hospitals_per_county.dropna(inplace=True)
hospitals_per_county= hospitals_per_county[hospitals_per_county["FIPS"] != "NOT AVAILABLE"]
hospitals_per_county["FIPS"] = hospitals_per_county["FIPS"].astype("int64")

hospitals_per_county.head()

In [ ]:
hospitals_per_county = hospitals_per_county.merge(average_hospital_rating_per_county, on="FIPS", how="left")
hospitals_per_county.head()

In [ ]:
county_time_series.head()

In [ ]:
crosswalk = crosswalk[['FIPS', 'CountyName', 'StateName']]
crosswalk["FIPS"] = crosswalk["FIPS"].astype("int64")
crosswalk.head()

In [ ]:
house_prices = county_time_series.groupby("RegionName").mean()
team_est = house_prices["ZHVI_AllHomes"].to_frame()
team_est.reset_index(level=0, inplace=True)
team_est.rename(index=str, columns={"RegionName": "FIPS", "ZHVI_AllHomes": "AverageHousePrice"}, inplace=True)
team_est["FIPS"] = team_est["FIPS"].astype("int64")
team_est.dropna(inplace=True)

team_est.head()

In [ ]:
public_schools_per_county = public_schools['COUNTYFIPS'].value_counts().to_frame()
public_schools_per_county.reset_index(level=0, inplace=True)
public_schools_per_county.rename(index=str, columns={"index": "FIPS", "COUNTYFIPS": "NumberOfSchools"}, inplace=True)
public_schools_per_county["FIPS"] = public_schools_per_county["FIPS"].astype("int64")


public_schools_per_county.head()

In [ ]:
team_est = team_est.merge(crosswalk, on="FIPS")

team_est.head()

In [ ]:
team_est = team_est.merge(unemployment_per_county, on="FIPS")
team_est.head()

In [ ]:
team_est = team_est.merge(public_schools_per_county, on="FIPS")
team_est.head()

In [ ]:
team_est = team_est.merge(hospitals_per_county, on="FIPS")
team_est.head()

dataset containing all the required features for the housing prices

In [ ]:
team_est = team_est[['FIPS', 'CountyName', 'StateName', 'NumberOfSchools', 'NumberOfHospitals', 'AverageHospitalRating', 'UnemploymentRate', 'AverageHousePrice']]
team_est.head()

In [ ]:
corrmat = team_est.corr()
plt.subplots(figsize=(15,15))
sns.heatmap(corrmat.abs(), vmax=.4, square=True)

In [ ]:
corrmat

There is a positive correlation between the averagehouseprice and the numberofschools and numberofhospital in a particular county and negative correlation between averagehouseprice and unemployment . So if a area has more schools and hospital then the house prices tend to go up. Also rating of hospital has no effect on the averagehouseprice. 

State with most house prices 